In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import re
import json
import requests
import time

In [3]:
review_df = pd.read_csv("data/steam_review_data.csv")
game_df = pd.read_csv("data/steamspy_data.csv")

In [4]:
review_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2117312 entries, 0 to 2117311
Data columns (total 18 columns):
 #   Column                          Dtype  
---  ------                          -----  
 0   Unnamed: 0                      int64  
 1   appid                           int64  
 2   recommendationid                int64  
 3   author                          int64  
 4   author_playtime_forever         int64  
 5   author_playtime_last_two_weeks  int64  
 6   author_last_played              int64  
 7   author_num_reviews              int64  
 8   votes_up                        int64  
 9   votes_funny                     int64  
 10  weighted_vote_score             float64
 11  comment_count                   int64  
 12  timestamp_created               int64  
 13  timestamp_updated               int64  
 14  steam_purchase                  bool   
 15  received_for_free               bool   
 16  written_during_early_access     bool   
 17  review                     

In [5]:
game_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63968 entries, 0 to 63967
Data columns (total 20 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   appid            63968 non-null  int64  
 1   name             63865 non-null  object 
 2   developer        56704 non-null  object 
 3   publisher        56711 non-null  object 
 4   score_rank       48 non-null     float64
 5   positive         63968 non-null  int64  
 6   negative         63968 non-null  int64  
 7   userscore        63968 non-null  int64  
 8   owners           63968 non-null  object 
 9   average_forever  63968 non-null  int64  
 10  average_2weeks   63968 non-null  int64  
 11  median_forever   63968 non-null  int64  
 12  median_2weeks    63968 non-null  int64  
 13  price            56854 non-null  float64
 14  initialprice     56856 non-null  float64
 15  discount         56856 non-null  float64
 16  languages        56814 non-null  object 
 17  genre       

In [7]:
game_df[game_df.developer.isna()]

,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,languages,genre,ccu,tags
226,8040,Championship Manager 2007,NaN,NaN,NaN,0,0,0,"0 .. 20,000",0,0,0,0,0.0,0.0,0.0,NaN,NaN,19,[]
227,8060,Championship Manager 2008,NaN,NaN,NaN,0,0,0,"0 .. 20,000",0,0,0,0,0.0,0.0,0.0,NaN,NaN,50,[]
264,9730,Tycoon City: New York,NaN,Retroism,NaN,84,135,0,"20,000 .. 50,000",71,0,71,0,999.0,999.0,0.0,English,NaN,2,"{'Simulation': 47, 'Management': 21, 'City Bui..."
309,11390,Crash Time 2,NaN,Meridian4,NaN,1320,1036,0,"100,000 .. 200,000",26,0,29,0,499.0,499.0,0.0,"English, French, German",NaN,3,"{'Combat Racing': 249, 'Racing': 242, 'Sandbox..."
598,29600,Guild Wars: Game of the Year,NaN,NaN,NaN,7,0,0,"0 .. 20,000",0,0,0,0,0.0,0.0,0.0,NaN,NaN,0,"{'RPG': 34, 'Massively Multiplayer': 34, 'MMOR..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63963,1899430,NaN,NaN,NaN,NaN,0,0,0,"0 .. 20,000",0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,[]
63964,1899990,GEHENA,NaN,NaN,NaN,0,0,0,"0 .. 20,000",0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,[]
63965,1900820,NaN,NaN,NaN,NaN,0,0,0,"0 .. 20,000",0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,[]
63966,1901700,TRADER LIFE SIMULATOR 2,NaN,NaN,NaN,0,0,0,"0 .. 20,000",0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,[]
